[View in Colaboratory](https://colab.research.google.com/github/castellwhite/MoviesClassification/blob/master/Stacking_Model.ipynb)

<h2>
# Movies Classification Project
</center></h4> <h4>
 <img src="http://usblogs.pwc.com/emerging-technology/wp-content/uploads/2017/07/deep-learning-900x280.png" >


# Machine Learning

</h4>

Developed by [Sergio Castelblanco](http://www.linkedin.com/in/sergio-castelblanco/) - [Jesus Solano](http://www.jesussolano.com)

Andes University

version 2.1, July 9 2018

In [28]:
!pip install tqdm
!pip install livelossplot

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [63]:
dataTraining = pd.read_csv('https://github.com/castellwhite/MoviesClassification/blob/master/datasets/dataTraining.csv?raw=true', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv("https://github.com/castellwhite/MoviesClassification/blob/master/datasets/dataTesting.csv?raw=true", encoding='UTF-8', index_col=0)
dataTesting.shape

(3383, 3)

In [0]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm
import os
%matplotlib inline

In [33]:
rm -r*

rm: invalid option -- '*'
Try 'rm --help' for more information.


# Read Data

In [30]:
# If datasets already exist do nothing. 


if not os.path.exists('dataTesting.csv'):
  ! wget https://raw.githubusercontent.com/castellwhite/MoviesClassification/master/datasets/dataTesting.csv
    
if not os.path.exists('dataTraining.csv'):
  ! wget https://raw.githubusercontent.com/castellwhite/MoviesClassification/master/datasets/dataTraining.csv
    

if not os.path.exists('images_resize_color.zip'):
  # Install the PyDrive wrapper & import libraries.
  # This only needs to be done once per notebook.
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  # Authenticate and create the PyDrive client.
  # This only needs to be done once per notebook.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Download a file based on its file ID(Grey).
  #
  # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
  file_id = '1ZzSvFeCsLVW4hPWqyf6jW0rAvyQt1L7J'
  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(downloaded['title'])
  !unzip -q images_resize_gray.zip 

  # Download a file based on its file ID(Color).
  #
  # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
  # id 160x256 95 : 1ZO1wPtxWfnthbm2NrXt4ca1ZvuxQWD_r
  #file_id = '1hAOuBMKUJDiyaHz6fVkwwSAHkmEVNvmP'
  
  #file_id = '1ZO1wPtxWfnthbm2NrXt4ca1ZvuxQWD_r'
  #downloaded = drive.CreateFile({'id': file_id})
  #downloaded.GetContentFile(downloaded['title'])
  #!unzip -q images_resize_color.zip 

dataTraining = pd.read_csv("dataTraining.csv",index_col=0)
dataTesting = pd.read_csv("dataTesting.csv",index_col=0)



You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
replace images_resize_gray/11_resize_gray.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [64]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [65]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# Load Images

In [66]:
# Load all grey images.

print('\nLoading images for traning...')

trainingImagesGrey = []
for i in tqdm(dataTraining.index):
    trainingImagesGrey.append(io.imread(os.path.join('images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    
trainingImagesGrey = np.stack(trainingImagesGrey)

print('\nLoading images for testing...')

testingImagesGrey = []
for i in tqdm(dataTesting.index):
    testingImagesGrey.append(io.imread(os.path.join('images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())

testingImagesGrey = np.stack(testingImagesGrey)

  2%|▏         | 175/7895 [00:00<00:04, 1746.79it/s]


Loading images for traning...


  5%|▍         | 166/3383 [00:00<00:01, 1650.86it/s]


Loading images for testing...


100%|██████████| 3383/3383 [00:02<00:00, 1572.99it/s]


In [0]:
# Apply PCA to soft analysis.

pca = PCA(n_components=32)

trainingImagesGrey_pca = pca.fit_transform(trainingImagesGrey)
testingImagesGrey_pca = pca.transform(testingImagesGrey)

In [0]:
# Get labels for each image. 

dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
yGenres = le.fit_transform(dataTraining['genres'])

In [0]:
# Creates datasets for training and testing.   
xTrainImaGrey, xValidationImaGrey, yTrainImaGrey, yValidationImaGrey = train_test_split(trainingImagesGrey_pca, yGenres, test_size=0.2, random_state=22)

# Machine Learning Images Models

### Random Forest Classifier

In [69]:
# Creates a random forest classifier. 

mcRandomIma = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=20, random_state=22,verbose=0))

# Train the model. 

mcRandomIma.fit(xTrainImaGrey,  yTrainImaGrey)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=22, verbose=0, warm_start=False),
          n_jobs=1)

In [70]:
# Score the AUC over the test set. 

y_pred_genres = mcRandomIma.predict_proba(xValidationImaGrey)

mcRandomIma_AUC=roc_auc_score(yValidationImaGrey, y_pred_genres, average='macro')

print('Random Forest model score is: ' , mcRandomIma_AUC , 'AUC.')

Random Forest model score is:  0.5696614483536034 AUC.


### Gradient Boost Classifier

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

# Creates a random forest classifier. 

mcGradientIma = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, max_depth=20, random_state=22,verbose=0))

# Train the model. 

mcGradientIma.fit(xTrainImaGrey,  yTrainImaGrey)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=20,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=22, subsample=1.0, verbose=0,
              warm_start=False),
          n_jobs=1)

In [20]:
# Score the AUC over the test set. 

y_pred_genres = mcGradientIma.predict_proba(xValidationImaGrey)

mcGradientIma_AUC=roc_auc_score(yValidationImaGrey, y_pred_genres, average='macro')

print('Random Forest model score is: ' , mcGradientIma_AUC , 'AUC.')

Random Forest model score is:  0.5579683680093176 AUC.


# Read Text

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

maxWords= 5000

vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=maxWords)

tokenText = vectorizer.fit_transform(dataTraining['plot'])
testingText = vectorizer.fit_transform(dataTesting['plot'])
tokenText.shape

(7895, 5000)

In [72]:
# Creates datasets for training and testing.   
xTrainTxt, xValidationTxt, yTrainTxt, yValidationTxt = train_test_split(tokenText, yGenres, test_size=0.20, random_state=22)
xTrainTxt.shape

(6316, 5000)

# Machine Learning  Text Models

### Random Forest

In [73]:
# Creates a random forest classifier. 

mcRandomTxt = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=22,verbose=0))

# Train the model. 

mcRandomTxt.fit(xTrainTxt,  yTrainTxt)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=22, verbose=0, warm_start=False),
          n_jobs=1)

In [74]:
# Score the AUC over the test set. 

y_pred_genres = mcRandomTxt.predict_proba(xValidationTxt)

mcRandomTxt_AUC = roc_auc_score(yValidationTxt, y_pred_genres, average='macro')

print('Random Forest model score is: ' ,mcRandomTxt_AUC, 'AUC.')

yPredTestRF = mcRandomTxt.predict_proba(testingText)

Random Forest model score is:  0.8155028010105557 AUC.


In [0]:
# Print prediction for test (kaggle) data. 

yPredTestRF = mcRandomTxt.predict_proba(testingText)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(yPredTest, index=dataTesting.index, columns=cols).to_csv('pred_genres_RF_Text.csv', index_label='ID')

print('Results were printed.....')

Results were printed.....


### Gradient Boosting

In [43]:
from sklearn.ensemble import GradientBoostingClassifier

# Creates a GradientBoostingClassifier. 

mcGradientTxt = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, max_depth=10, random_state=22,verbose=0))

# Train the model. 

mcGradientTxt.fit(xTrainTxt,  yTrainTxt)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=22, subsample=1.0, verbose=0,
              warm_start=False),
          n_jobs=1)

In [44]:
# Score the AUC over the test set. 

y_pred_genres = mcGradientTxt.predict_proba(xValidationTxt)

mcGradientTxt_AUC = roc_auc_score(yValidationTxt, y_pred_genres, average='macro')

print('Gradient Boosting score is: ' ,mcGradientTxt_AUC, 'AUC.')

yPredTestGB = mcGradientTxt.predict_proba(testingText)

Gradient Boosting score is:  0.7651230922952764 AUC.


In [0]:
# Print prediction for test (kaggle) data. 

yPredTestGB = mcGradientTxt.predict_proba(testingText)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(yPredTest, index=dataTesting.index, columns=cols).to_csv('pred_genres_GB_Text.csv', index_label='ID')

print('Results were printed.....')

Results were printed.....


In [0]:
xtest=  np.c_[yPredTestRF,yPredTestGB]
xtest.shape

## Stacking Model

In [0]:
from sklearn.ensemble import ExtraTreesClassifier
from google.colab import files
#predGbText= mcGradientTxt.predict_proba(xValidationTxt)
predRfText= mcRandomTxt.predict_proba(xValidationTxt)
y_pred_genres = mcRandomIma.predict_proba(xValidationImaGrey)
yPredTestImaRF= mcRandomIma.predict_proba(testingImagesGrey_pca)


mixText =  np.c_[predRfText,y_pred_genres]
Xtest =  np.c_[yPredTestRF,yPredTestImaRF]

clf_etc = OneVsRestClassifier(ExtraTreesClassifier(n_jobs=-1, n_estimators=400, max_depth=6, random_state=42))
clf_etc.fit(mixText, yValidationTxt)
y_pred_genres_etc = clf_etc.predict_proba(Xtest)
#acc_etc=roc_auc_score(yValidationTxt, y_pred_genres_etc, average='macro')

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(y_pred_genres_etc, index=dataTesting.index, columns=cols).to_csv('pred_genres_Stack1.csv', index_label='ID')
#print("Stacking  AUC:",acc_etc)

files.download('pred_genres_Stack1.csv')

In [0]:
from keras import backend as K
K.clear_session()

mixModel = Sequential()
mixModel.add(Dense(64, input_shape=(mixText.shape[1],)))
mixModel.add(Activation('tanh'))
#mixModel.add(BatchNormalization())
mixModel.add(Dropout(0.5))
mixModel.add(Dense(24, activation='sigmoid'))
mixModel.summary()

mixModel.compile(loss='categorical_crossentropy',
              optimizer=opts.adam(lr=0.001), 
                 metrics=["accuracy"])

batch_size = 5000

mixModel.fit(mixText,yValidationStack, 
             epochs = 100,
             verbose = 2,
             batch_size=batch_size, 
             validation_data=[xValidationTxt,yValidationTxt],
             callbacks=[PlotLossesKeras()])

mixModel.save('mixNNtrained.h5')
